In [11]:
# Connected Components Labeling: Use a labeling function to identify and group connected pixels. You can filter out labels with fewer than 480 pixels and reassign these to the nearest larger region.

In [12]:
import os
import numpy as np
import rasterio
from scipy.ndimage import label, binary_dilation

In [14]:
# Define directories
input_dir = "../predictions/relabeled_output_with_soils"
output_dir = "../predictions/dissolved_output_with_soils"
os.makedirs(output_dir, exist_ok=True)

# Define pixel threshold for 2m² area
pixel_threshold = 128  # Approximate pixel count for 2m²

def iterative_dilation_fill(data, mask, max_iterations=10):
    """Apply an iterative dilation fill to propagate neighboring values into empty cells."""
    for _ in range(max_iterations):
        for value in np.unique(data):
            if value <= 0:
                continue
            value_mask = (data == value)
            dilated_mask = binary_dilation(value_mask, structure=np.ones((3, 3)))
            fill_mask = dilated_mask & mask
            data[fill_mask] = value
        mask = (data <= 0)
        if not np.any(mask):
            break
    return data

def process_raster_with_dilation(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype(np.int32)
        profile = src.profile
        unique_values = np.unique(data)
        print(f"Processing {input_path} with unique values: {unique_values}")

        processed_data = data.copy()

        for value in unique_values:
            if value == 0:
                continue
            class_mask = (data == value)
            labeled_class, num_features = label(class_mask)
            print(f"  Value {value}: {num_features} regions found")
            for region_id in range(1, num_features + 1):
                region_mask = (labeled_class == region_id)
                region_size = np.sum(region_mask)
                if region_size < pixel_threshold:
                    processed_data[region_mask] = -9999
                else:
                    processed_data[region_mask] = value

        empty_mask = (processed_data == -9999)
        filled_data = iterative_dilation_fill(processed_data, empty_mask)
        filled_data[filled_data == -9999] = 0

        # Update profile with compression, dtype, and tiling
        profile.update(dtype=rasterio.uint16, compress='LZW', blockxsize=256, blockysize=256, tiled=True)
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(filled_data.astype(rasterio.uint16), 1)

def batch_process_rasters(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            print(f"Processing file: {filename}")
            process_raster_with_dilation(input_path, output_path)
            print(f"Saved processed file to: {output_path}")

# Run the batch processing
batch_process_rasters(input_dir, output_dir)


Processing file: SK1054_combined.tif
Processing ../predictions/relabeled_output_with_soils/SK1054_combined.tif with unique values: [101 102 103 104 201 202 203 302 303 401 403 404 409 410 411 504]
  Value 101: 857 regions found
  Value 102: 389 regions found
  Value 103: 138 regions found
  Value 104: 316 regions found
  Value 201: 176 regions found
  Value 202: 104 regions found
  Value 203: 64 regions found
  Value 302: 288 regions found
  Value 303: 590 regions found
  Value 401: 2 regions found
  Value 403: 7 regions found
  Value 404: 2 regions found
  Value 409: 2 regions found
  Value 410: 20 regions found
  Value 411: 5 regions found
  Value 504: 415 regions found
Saved processed file to: ../predictions/dissolved_output_with_soils/SK1054_combined.tif
Processing file: SJ9758_combined.tif
Processing ../predictions/relabeled_output_with_soils/SJ9758_combined.tif with unique values: [101 102 103 104 201 202 203 204 302 303 403 404 409 410 411 504]
  Value 101: 769 regions found
  V